In [3]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [4]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

In [5]:
def answer_one():
    x = 0
    y = 0
    
    for what in spam_data['target']:
        if what==1: x += 1
        else: y += 1
    
    return (x/(x+y))*100.0

answer_one()

13.406317300789663

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vect = CountVectorizer().fit(X_train)
    
    l = 0
    ans = ""
    
    for token in vect.get_feature_names():
        if len(token)>l:
            l = len(token)
            ans = token
    
    return ans

answer_two()

'com1win150ppmx3age16subscription'

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    model = MultinomialNB(alpha = 0.1)
    model.fit(X_train_vectorized, y_train)
    
    predictions = model.predict(vect.transform(X_test))
    
    ans = roc_auc_score(y_test, predictions)
    
    return ans

answer_three()

0.9720812182741116

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    vect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    feature_names = np.array(vect.get_feature_names())
    
    sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
    
    sorted_tfidf_features = feature_names[sorted_tfidf_index]
    sorted_tfidf_value = (X_train_vectorized.max(0).toarray()[0])[sorted_tfidf_index]
    
    overall = list(zip(sorted_tfidf_features, sorted_tfidf_value))
    
    one = overall[:20]
    two = overall[:-21:-1]
    
    answer_one = pd.Series([x[1] for x in one], index = [x[0] for x in one])
    answer_two = pd.Series([x[1] for x in two], index = [x[0] for x in two])
    
    return (answer_one, answer_two)

answer_four()

(sympathetic     0.074475
 healer          0.074475
 aaniye          0.074475
 dependable      0.074475
 companion       0.074475
 listener        0.074475
 athletic        0.074475
 exterminator    0.074475
 psychiatrist    0.074475
 pest            0.074475
 determined      0.074475
 chef            0.074475
 courageous      0.074475
 stylist         0.074475
 psychologist    0.074475
 organizer       0.074475
 pudunga         0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64,
 146tf150p    1.000000
 havent       1.000000
 home         1.000000
 okie         1.000000
 thanx        1.000000
 er           1.000000
 anything     1.000000
 lei          1.000000
 nite         1.000000
 yup          1.000000
 thank        1.000000
 ok           1.000000
 where        1.000000
 beerage      1.000000
 anytime      1.000000
 too          1.000000
 done         1.000000
 645          1.000000
 tick         0.980166
 blank        0.932702
 dt

In [9]:
def answer_five():
    vect = TfidfVectorizer(min_df = 3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    model = MultinomialNB(alpha = 0.1)
    model.fit(X_train_vectorized, y_train)
    
    predictions = model.predict(vect.transform(X_test))
    
    ans = roc_auc_score(y_test, predictions)
    
    return ans

answer_five()

0.9416243654822335

In [10]:
def answer_six():
    spam = 0
    not_spam = 0
    spam_cn = 0
    not_spam_cn = 0
    
    for msg, what in zip(spam_data['text'], spam_data['target']):
        if what==1:
            spam += len(msg)
            spam_cn += 1
        else :
            not_spam += len(msg)
            not_spam_cn += 1
    
    return (not_spam/not_spam_cn, spam/spam_cn)

answer_six()

(71.02362694300518, 138.8661311914324)

In [12]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [13]:
from sklearn.svm import SVC

def answer_seven():
    len_train = [len(msg) for msg in X_train]
    len_test =  [len(msg) for msg in X_test]
    
    vect = TfidfVectorizer(min_df = 5).fit(X_train)
    
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    X_train_vectorized = add_feature(X_train_vectorized, len_train)
    X_test_vectorized = add_feature(X_test_vectorized, len_test)
    
    model = SVC(C = 10000)
    model.fit(X_train_vectorized, y_train)
    
    predictions = model.predict(X_test_vectorized)
    
    ans = roc_auc_score(y_test, predictions)
    
    return ans

answer_seven()

0.9661689557407943

In [14]:
import nltk

def answer_eight():
    spam = 0
    not_spam = 0
    spam_cn = 0
    not_spam_cn = 0
    
    for msg, what in zip(spam_data['text'], spam_data['target']):
        if what==1:
            for c in msg:
                if c.isnumeric(): spam += 1
                    
            spam_cn += 1
        else :
            for c in msg:
                if c.isnumeric(): not_spam += 1
                    
            not_spam_cn += 1
    
    return (not_spam/not_spam_cn, spam/spam_cn)

answer_eight()

(0.2992746113989637, 15.76037483266399)

In [15]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
#     len_train = [len(msg) for msg in X_train]
#     len_test =  [len(msg) for msg in X_test]
    
    num_train = [sum(c.isnumeric() for c in msg) for msg in X_train]
    num_test =  [sum(c.isnumeric() for c in msg) for msg in X_test]
    
    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
#     X_train_vectorized = add_feature(X_train_vectorized, len_train)
#     X_test_vectorized = add_feature(X_test_vectorized, len_test)
    
    X_train_vectorized = add_feature(X_train_vectorized, num_train)
    X_test_vectorized = add_feature(X_test_vectorized, num_test)
    
    model = LogisticRegression(C = 100)
    model.fit(X_train_vectorized, y_train)
    
    predictions = model.predict(X_test_vectorized)
    
    ans = roc_auc_score(y_test, predictions)
    
    return ans

answer_nine()

/home/sayem/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9678709064054463

In [16]:
def answer_ten():
    spam = 0
    not_spam = 0
    spam_cn = 0
    not_spam_cn = 0
    
    for msg, what in zip(spam_data['text'], spam_data['target']):
        if what==1:
            for c in msg:
                if c.isnumeric()==False and c.isalpha()==False and c!='_': spam += 1
                    
            spam_cn += 1
        else :
            for c in msg:
                if c.isnumeric()==False and c.isalpha()==False and c!='_': not_spam += 1
                    
            not_spam_cn += 1
    
    return (not_spam/not_spam_cn, spam/spam_cn)
    
answer_ten()

(17.29181347150259, 29.041499330655956)

In [17]:
def answer_eleven():
    word_train = [sum((c.isnumeric()==False and c.isalpha()==False and c!='_') for c in msg) for msg in X_train]
    word_test =  [sum((c.isnumeric()==False and c.isalpha()==False and c!='_') for c in msg) for msg in X_test]
    
    len_train = [len(msg) for msg in X_train]
    len_test =  [len(msg) for msg in X_test]
    
    digit_train = [sum(c.isnumeric() for c in msg) for msg in X_train]
    digit_test =  [sum(c.isnumeric() for c in msg) for msg in X_test]
    
    vect = CountVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb').fit(X_train)
    
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    X_train_vectorized = add_feature(X_train_vectorized, [len_train, digit_train, word_train])
    X_test_vectorized = add_feature(X_test_vectorized, [len_test, digit_test, word_test])
    
    model = LogisticRegression(C = 100)
    model.fit(X_train_vectorized, y_train)
    
    predictions = model.predict(X_test_vectorized)
    
    area = roc_auc_score(y_test, predictions)
    
    feature_names = np.array(vect.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])

    sorted_coef_index = model.coef_[0].argsort()
    
    one = list(feature_names[sorted_coef_index[:10]])
    two = list(feature_names[sorted_coef_index[:-11:-1]])
    
    return (area, one, two)

answer_eleven()

/home/sayem/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.9809793219360643,
 ['..', '. ', ' i', ' y', ' go', '? ', 'ok', 'pe', 'he', 'go'],
 ['digit_count', 'co', 'ne', 'ww', 'xt', 'ia', 'ar', 'uk', ' ch', ' a '])